Plots saved at /home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_{metab}_results/{metab}/combined

Dots in regression plot are the averaged values in significant clusters for each subject.

In [ ]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
from mayavi import mlab
from surfer import Brain
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import subprocess
from pathlib import Path
from scipy import stats
from matplotlib.colors import LinearSegmentedColormap

def setup_freesurfer():
    """Initialize FreeSurfer environment"""
    freesurfer_home = '/home/localadmin/freesurfer'
    subjects_dir = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia"
    os.environ['FREESURFER_HOME'] = freesurfer_home
    os.environ['SUBJECTS_DIR'] = subjects_dir
    
    setup_cmd = f"bash -c 'source {freesurfer_home}/SetUpFreeSurfer.sh; env'"
    try:
        process = subprocess.Popen(setup_cmd, stdout=subprocess.PIPE, shell=True)
        output, _ = process.communicate()
        for line in output.decode().split('\n'):
            if '=' in line:
                key, value = line.split('=', 1)
                os.environ[key] = value
        print("FreeSurfer environment initialized successfully")
    except Exception as e:
        print(f"Error setting up FreeSurfer: {e}")
        raise

def find_cluster_file(glmdir_path, for_visualization=True):
    """Find appropriate cluster file in GLM directory
    for_visualization: if True, find masked file, if False, find cluster file"""
    if os.path.exists(glmdir_path):
        print(f"\nLooking in directory: {glmdir_path}")
        all_files = os.listdir(glmdir_path)
        print("All files:", all_files)
        
        if for_visualization:
            files = [f for f in all_files 
                    if 'perm' in f and 'cluster' in f and f.endswith('.mgh')]
            
        selected_file = files[0] if files else None
        print(f"Selected file: {selected_file}")
        return selected_file
    return None

def load_freesurfer_lut(filename):
    """Load a FreeSurfer-style colormap from file"""
    data = np.loadtxt(filename)
    rgb_data = data[:, :3]  # Take only RGB columns
    return LinearSegmentedColormap.from_list('custom', rgb_data)

def create_regression_plot(data_dict, metab_values, fig, ax, x_label=None):
    """Create regression plot for EP and HC groups"""
    sns.set_style(rc={'axes.facecolor': 'black',
                     'figure.facecolor': 'black',
                     'xtick.color': 'white',
                     'ytick.color': 'white',
                     'axes.edgecolor': 'white',
                     'axes.labelcolor': 'white',
                     'text.color': 'white'})
    
    colors = {'EP': '#E41A1C', 'HC': '#377EB8'}
    
    for group in ['EP', 'HC']:
        if group in data_dict and len(data_dict[group]) > 0:
            x = metab_values[group]
            y = data_dict[group]
            
            # Create scatter plot
            ax.scatter(x, y, c=colors[group], label=group, alpha=0.65, 
                      edgecolors='white', linewidth=1)
            
            # Add regression line
            z = np.polyfit(x, y, 1)
            p = np.poly1d(z)
            x_range = np.linspace(min(x), max(x), 100)
            ax.plot(x_range, p(x_range), c=colors[group], alpha=0.8)
    
    # Set x-axis label if provided
    if x_label:
        ax.set_xlabel(x_label, fontsize=20)  # Increased font size
    
    ax.legend(frameon=False, fontsize=18)  # Increased font size
    ax.set_facecolor('black')
    plt.setp(ax.spines.values(), color='white')
    
    # Increase font size for tick labels and ylabel
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.set_ylabel(ax.get_ylabel(), fontsize=20)  # Increased font size
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.2f}'))
    
    return ax

def fig_to_array(fig):
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    return data

def create_brain_views(subject_id, hemi, sig_file, temp_file_base, output_path, metab):
    """Create lateral and medial views of brain surface showing -log10(p-values)"""
    try:
        cluster_data = nib.load(sig_file).get_fdata().squeeze()
        thresh = 1.3
        views = ['lateral', 'medial']
        temp_files = []
        colorbar_files = []
        
        # Check if we have significant clusters
        has_sig_clusters = np.max(cluster_data) > thresh
        
        for view in views:
            temp_file = f"{temp_file_base}_{view}.png"
            temp_files.append(temp_file)
            
            brain = Brain(subject_id, hemi, 'inflated',
                         background="black",
                         cortex="low_contrast",
                         size=(800, 600))
            
            if has_sig_clusters:
                max_value = np.max(cluster_data)
                print(f"Maximum -log10(p-value): {max_value}")
                
                custom_cmap = load_freesurfer_lut('/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/results/nih_iso.cmap')
                
                brain.add_data(cluster_data,
                             min=thresh,
                             max=max_value,
                             mid=(max_value + thresh)/2,
                             thresh=thresh,
                             colormap='YlOrRd',
                             alpha=1.0,
                             smoothing_steps=0,
                             colorbar=True,
                             remove_existing=True)
                
                # Save colorbar in metabolite-specific directory
                colorbar_file = os.path.join(output_path,
                                           f"{os.path.basename(temp_file_base)}_{view}_colorbar.png")
                colorbar_files.append(colorbar_file)
                
                fig, ax = plt.subplots(figsize=(4, 0.5))
                norm = plt.Normalize(vmin=thresh, vmax=max_value)
                sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
                sm.set_array([])
                cbar = plt.colorbar(sm, cax=ax, orientation='horizontal')
                cbar.set_label('-log10(p-value)', color='white')
                cbar.ax.yaxis.set_tick_params(color='white')
                cbar.ax.tick_params(labelcolor='white')
                plt.savefig(colorbar_file, bbox_inches='tight', dpi=300, facecolor='black')
                plt.close(fig)
            
            # Always show the brain view, even without significant clusters
            brain.show_view(view, distance=350)
            brain.save_image(temp_file)
            # Save individual brain view
            final_brain_file = os.path.join(output_path, metab, 'individual',
                                          f'{os.path.basename(temp_file_base)}_{view}.png')
            brain.save_image(final_brain_file)
            
            mlab.close(brain._figures[0])
        
        return temp_files, colorbar_files, has_sig_clusters
    except Exception as e:
        print(f"Error creating brain views: {e}")
        return [], []
def calculate_pvalue(group1, group2, alternative='two-sided'):
    """
    Calculate p-value for the difference between two independent groups using t-test.
    
    Parameters:
    group1 (array-like): First group of observations
    group2 (array-like): Second group of observations
    alternative (str): The alternative hypothesis, one of 'two-sided', 'less', 'greater'
    
    Returns:
    float: p-value
    """
    # Perform independent t-test
    t_stat, p_value = stats.ttest_ind(group1, group2, alternative=alternative)
    
    return p_value   
def create_boxplot(data_dict, fig, ax):
    sns.set_style(rc={'axes.facecolor': 'black',
                     'figure.facecolor': 'black',
                     'xtick.color': 'white',
                     'ytick.color': 'white',
                     'axes.edgecolor': 'white',
                     'axes.labelcolor': 'white',
                     'text.color': 'white'})
    
    all_data = []
    labels = []
    # Store data for Cohen's d calculation
    ep_data = np.array(data_dict['EP']) if 'EP' in data_dict else None
    hc_data = np.array(data_dict['HC']) if 'HC' in data_dict else None
    
    for group in ['EP', 'HC']:
        if group in data_dict and len(data_dict[group]) > 0:
            values = np.array(data_dict[group])
            all_data.extend(values.flatten())
            labels.extend([group] * len(values.flatten()))
    
    plot_data = pd.DataFrame({
        'Group': labels,
        'Value': all_data
    })
    
    palette = {'EP': '#E41A1C', 'HC': '#377EB8'}
    bp = sns.boxplot(data=plot_data, x='Group', y='Value',
                    palette=palette, ax=ax)
    
    sns.swarmplot(data=plot_data, x='Group', y='Value',
                  color='white', alpha=0.65, ax=ax)
    
    # Calculate and display Cohen's d
    if ep_data is not None and hc_data is not None:
        cohens_d = calculate_cohens_d(ep_data.flatten(), hc_data.flatten())
        pvalue = calculate_pvalue(ep_data.flatten(), hc_data.flatten())
        
        # p-value first (higher position)
        plt.text(0.5, 0.90, f"p = {pvalue:.2g}", 
                transform=ax.transAxes, color='white',
                horizontalalignment='center', fontsize=20)  # Increased font size
        
        # Cohen's d below it
        plt.text(0.5, 0.80, f"Cohen's d = {cohens_d:.2g}", 
                transform=ax.transAxes, color='white',
                horizontalalignment='center', fontsize=20)  # Increased font size
    
    ax.set_xlabel('')
    ax.set_title('')
    ax.set_facecolor('black')
    plt.setp(ax.spines.values(), color='white')
    
    # Increase font size for tick labels
    ax.tick_params(axis='both', which='major', labelsize=18)
    
    # Increase font size for legend
    plt.legend(fontsize=16)
    
    # Increase font size for y-label
    ax.set_ylabel(ax.get_ylabel(), fontsize=20)
    
    return bp

def calculate_cohens_d(group1, group2):
    """Calculate Cohen's d effect size between two groups."""
    n1, n2 = len(group1), len(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    
    # Pooled standard deviation
    pooled_se = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
    
    # Cohen's d
    cohens_d = (np.mean(group1) - np.mean(group2)) / pooled_se
    return cohens_d

def process_parameter(base_path, param, contrast, metab, output_path, groups_data, letter, counter):
    """
    Process neuroimaging parameters and create visualizations for brain regions with significant clusters.
    
    Parameters:
    base_path (str): Base directory path
    param (str): Parameter to analyze (e.g., 'md', 'fa')
    contrast (str): Contrast type (e.g., 'EP_metab_negative')
    metab (str): Metabolite type
    output_path (str): Output directory path
    groups_data (pd.DataFrame): DataFrame containing group data
    letter (str): Letter label for the plot
    counter (int): Counter for plot organization
    """
    metab_output_path = os.path.join(output_path, metab)
    temp_dir = os.path.join(metab_output_path, 'temp')
    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(os.path.join(metab_output_path, 'combined'), exist_ok=True)
    os.makedirs(os.path.join(metab_output_path, 'individual'), exist_ok=True)
    
    hemis = ['lh', 'rh']
    view_order = [(0, 'lateral'), (0, 'medial'), (1, 'medial'), (1, 'lateral')]
    
    try:
        brain_images = []
        brain_data = {hemi: {} for hemi in hemis}
        significant_data = []  # Collection of significant data per hemisphere
        
        # Initialize data dictionaries
        data_by_group = {'EP': None, 'HC': None}
        thickness_by_group = {'EP': None, 'HC': None}
        metab_by_group = {'EP': None, 'HC': None}
        
        # Collect brain data paths for each hemisphere
        for hemi in hemis:
            glmdir = os.path.join(base_path, 'new_tcho_results',
                               f'group_{hemi}_{param}_fwhm6_demeaned_{contrast}.glmdir',
                               f'{contrast}')
            
            try:
                all_files = os.listdir(glmdir)
                cluster_files = [f for f in all_files 
                               if 'perm' in f and 'cluster' in f and f.endswith('.mgh')]
                if not cluster_files:
                    print(f"No masked files found in {glmdir}")
                    continue
                    
                cluster_file_name = cluster_files[0]
                
                brain_data[hemi] = {
                    'cluster': os.path.join(glmdir, cluster_file_name),
                    'param': os.path.join(base_path, 'new_group_analysis', f'group_{hemi}_{param}.mgh'),
                    'thickness': os.path.join(base_path, 'new_group_analysis', f'group_{hemi}_thickness.mgh')
                }
            except Exception as e:
                print(f"Error processing hemisphere {hemi}: {e}")
                continue
        
        # Process brain views and collect significant data
        for hemi_idx, view in view_order:
            hemi = hemis[hemi_idx]
            if hemi not in brain_data:
                continue
                
            temp_file_base = os.path.join(temp_dir, f'{contrast}_{param}_{hemi}')
            temp_files, colorbar_files, has_sig_clusters = create_brain_views(
                'fsaverage', hemi, 
                brain_data[hemi]['cluster'], 
                temp_file_base,
                output_path,
                metab
            )
            
            if temp_files:
                img = plt.imread(temp_files[0 if view == 'lateral' else 1])
                brain_images.append(img)
            
            if view == 'lateral' and has_sig_clusters:
                try:
                    cluster_data = nib.load(brain_data[hemi]['cluster']).get_fdata()
                    sig_vertices = cluster_data > 1.3
                    param_data = nib.load(brain_data[hemi]['param']).get_fdata()
                    thickness_data = nib.load(brain_data[hemi]['thickness']).get_fdata()
                    significant_data.append((sig_vertices, param_data, thickness_data))
                except Exception as e:
                    print(f"Error collecting data for {hemi}: {e}")
        
        # Process collected data once for all significant clusters
        if significant_data:
            for group in ['EP', 'HC']:
                group_indices = groups_data['group'] == group
                group_values = []
                thickness_values = []
                
                # Store metabolite values once per group
                metab_by_group[group] = groups_data[group_indices][metab].values
                
                for sig_vertices, param_data, thickness_data in significant_data:
                    significant_values = param_data[sig_vertices]
                    significant_thickness = thickness_data[sig_vertices]
                    group_data = np.mean(significant_values, axis=0)[group_indices]
                    group_thickness = np.mean(significant_thickness, axis=0)[group_indices]
                    group_values.append(group_data)
                    thickness_values.append(group_thickness)
                
                if group_values:
                    # Average across hemispheres if needed
                    data_by_group[group] = np.mean(group_values, axis=0)
                    thickness_by_group[group] = np.mean(thickness_values, axis=0)
        
        # Create visualization plots if we have data
        if any(v is not None for v in data_by_group.values()):
            # Create regression plot
            reg_fig = plt.figure(figsize=(8, 6))
            reg_fig.patch.set_facecolor('black')
            reg_ax = reg_fig.add_subplot(111)
            create_regression_plot(data_by_group, metab_by_group, reg_fig, reg_ax,
                                x_label=rf'{"Myo-Ins" if metab == "ins" else metab.capitalize()}$_\mathregular{{PFC}}$ [$\mu$mol/gram]')
            plt.ylabel(f'{param.upper()}')
            reg_img = fig_to_array(reg_fig)
            plt.close(reg_fig)
            brain_images.append(reg_img)
            
            # Create thickness boxplot
            box_fig = plt.figure(figsize=(8, 6))
            box_fig.patch.set_facecolor('black')
            box_ax = box_fig.add_subplot(111)
            create_boxplot(thickness_by_group, box_fig, box_ax)
            plt.ylabel('Thickness (mm)')
            box_img = fig_to_array(box_fig)
            plt.close(box_fig)
            brain_images.append(box_img)
        
        # Create combined figure
        if brain_images:
            fig, axes = plt.subplots(1, len(brain_images), figsize=(38, 8))
            plt.subplots_adjust(wspace=0, hspace=0)
            
            for idx, (ax, img) in enumerate(zip(axes, brain_images)):
                ax.imshow(img)
                ax.axis('off')
                
                if idx == 3:
                    ax.text(0.8, 0.9, 'R', color='white', fontsize=18,
                            transform=ax.transAxes)
                elif idx == 0:
                    contrast_labels = {
                        'med_HCEP': f'{letter}) {param.upper()}: {metab.capitalize()} $\\beta_{{HC}} > \\beta_{{EP}}$',
                        'med_EPHC': f'{letter}) {param.upper()}: {metab.capitalize()} $\\beta_{{EP}} > \\beta_{{HC}}$',
                        'EP_metab_negative': f'{letter}) {param.upper()}: {metab.capitalize()} $\\beta_{{EP}} < 0$',
                        'EP_metab_positive': f'{letter}) {param.upper()}: {metab.capitalize()} $\\beta_{{EP}} > 0$'
                    }
                    
                    if contrast in contrast_labels:
                        ax.text(0.1, 0.92, contrast_labels[contrast],
                               color='black', fontsize=18,
                               transform=ax.transAxes,
                               bbox=dict(facecolor='white', 
                                       alpha=1.0,
                                       edgecolor='none',
                                       pad=3))
            
            plt.savefig(os.path.join(metab_output_path, 'combined', 
                                    f'{contrast}_{param}_combined.png'),
                        facecolor='black', bbox_inches='tight', dpi=300, pad_inches=0.1)
            plt.close()
        
    except Exception as e:
        print(f"Error processing {param} - {contrast}: {e}")
    finally:
        if os.path.exists(temp_dir):
            for file in os.listdir(temp_dir):
                try:
                    os.remove(os.path.join(temp_dir, file))
                except Exception as e:
                    print(f"Error removing temp file {file}: {e}")
            try:
                os.rmdir(temp_dir)
            except Exception as e:
                print(f"Error removing temp directory: {e}")
                

def main():
    base_path = '/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia'
    output_path = os.path.join(base_path, 'new_tcho_results')
    os.makedirs(output_path, exist_ok=True)
    
    # Setup FreeSurfer
    setup_freesurfer()
    
    # Load group data
    groups_data = pd.read_csv(os.path.join(base_path, 
                             'MRS_data_curated_+paths+avg_skeleton+registration.csv'))
    
    # Define specific parameter-contrast combinations for each metabolite
    metabolite_analyses = {
        # 'ins': [
        #     ('fa', 'EP_metab_negative'),
        #     ('fa', 'med_HCEP')
        # ],
        
        'tcho': [
            ('fa', 'EP_metab_negative'),
            ('fa', 'med_HCEP'),
            ('md', 'EP_metab_positive'),
            ('md', 'EP_metab_negative'),
            ('md', 'med_EPHC')
        ],
    }
    
    # Create a list of letters for counters
    letters = ['A', 'B', 'C', 'D', 'E']  # Add more if needed
    
    # Process each metabolite and its parameter-contrast combinations
    for metab, analyses in metabolite_analyses.items():
        print(f"\nProcessing {metab} analyses...")
        # Create metabolite-specific output directories
        # os.makedirs(os.path.join(output_path, 'combined'), exist_ok=True)
        # os.makedirs(os.path.join(output_path, 'individual'), exist_ok=True)
        # os.makedirs(os.path.join(output_path, 'temp'), exist_ok=True)
        
        counter = 0
        
        # Process each parameter-contrast combination for this metabolite
        for param, contrast in analyses:
            print(f"Creating visualizations for {param} - {contrast}")
            process_parameter(base_path, param, contrast, metab, output_path, groups_data, letters[counter], counter)
            counter += 1

if __name__ == "__main__":
    main()